In [1]:
from OneDimSpiralHelpers import spiral_curve, plot_spiral, compute_true_tangent_spiral, plot_spiral_with_tangents, compute_true_normal_spiral, compute_knn_neighbors, estimate_tangent_normal_pca
import numpy as np
from OneDimSprialMetrics import evaluate_tangent_normal_errors

In [2]:
# 1. Generate data
t = np.linspace(0, 4*np.pi, 1000)
X = spiral_curve(t, a=0.5, b=0.2)

# 2. True tangent/normal
T_true = compute_true_tangent_spiral(t, a=0.5, b=0.2)
N_true = compute_true_normal_spiral(t, a=0.5, b=0.2)

# 3. Visualize true tangents/normals
fig = plot_spiral_with_tangents(
    X,
    T_true,
    N=N_true,
    every_n=100,
    scale_tangent=0.6,
    scale_normal=0.6,
    title="Spiral with True Tangents and Normals",
)
fig.show()

# 4. Function-free estimates
neighbors_idx = compute_knn_neighbors(X, k=5)
T_est, N_est = estimate_tangent_normal_pca(X, neighbors_idx)

# 5. Visualize estimated tangents/normals
fig = plot_spiral_with_tangents(
    X,
    T_est,
    N=N_est,
    every_n=100,
    scale_tangent=0.6,
    scale_normal=0.6,
    title="Spiral with PCA-Estimated Tangents and Normals",
)
fig.show()

# 6. Metrics
results = evaluate_tangent_normal_errors(T_true, T_est, N_true, N_est)

print("Tangent stats:", results["tangent"]["stats"])
print("Normal stats:",  results["normal"]["stats"])


Tangent stats: {'mean_rad': 0.0059005305585433845, 'median_rad': 0.0057431677657156215, 'p90_rad': 0.006005031303238576, 'max_rad': 0.04295809157480027, 'mean_deg': 0.3380754978925062, 'median_deg': 0.3290592740110839, 'p90_deg': 0.3440629495195148, 'max_deg': 2.4613173431725555}
Normal stats: {'mean_rad': 0.0059005305585364985, 'median_rad': 0.005743167765705956, 'p90_rad': 0.006005031303238576, 'max_rad': 0.04295809157480027, 'mean_deg': 0.33807549789211166, 'median_deg': 0.3290592740105301, 'p90_deg': 0.3440629495195148, 'max_deg': 2.4613173431725555}


In [3]:
import numpy as np

def run_spiral_knn_sweep(
    sample_sizes,
    k_values,
    a=0.5,
    b=0.2,
    t_min=0.0,
    t_max=4*np.pi,
    noise_sigma=0.0,
):
    """
    Sweep over sample sizes and k-NN values to evaluate tangent/normal estimation quality.

    Parameters
    ----------
    sample_sizes : list[int]
        List of N values (number of points on the spiral).
    k_values : list[int]
        List of k values (neighbors) for local PCA.
    a, b, t_min, t_max : float
        Spiral parameters and t-range for sampling.
    noise_sigma : float
        Std dev of isotropic Gaussian noise added to the spiral points.

    Returns
    -------
    results : dict
        {
          "sample_sizes": sample_sizes,
          "k_values": k_values,
          "mean_tangent_deg": np.ndarray shape (len(sample_sizes), len(k_values)),
          "mean_normal_deg":  np.ndarray shape (len(sample_sizes), len(k_values)),
          "full_stats": nested dict of all stats per (i, j)
        }
    """
    sample_sizes = list(sample_sizes)
    k_values = list(k_values)

    mean_tangent_deg = np.zeros((len(sample_sizes), len(k_values)))
    mean_normal_deg  = np.zeros((len(sample_sizes), len(k_values)))

    full_stats = {}  # (N, k) -> stats dict

    for i, N in enumerate(sample_sizes):
        # 1. Sample spiral
        t = np.linspace(t_min, t_max, N)
        X = spiral_curve(t, a=a, b=b)

        if noise_sigma > 0:
            X = X + np.random.randn(*X.shape) * noise_sigma

        # 2. True tangent/normal
        T_true = compute_true_tangent_spiral(t, a=a, b=b)
        N_true = compute_true_normal_spiral(t, a=a, b=b)

        for j, k in enumerate(k_values):
            # Safety: k must be < N
            if k >= N:
                raise ValueError(f"k={k} must be < N={N}")

            # 3. Function-free KNN + PCA
            neighbors_idx = compute_knn_neighbors(X, k=k)
            T_est, N_est = estimate_tangent_normal_pca(X, neighbors_idx)

            # 4. Metrics
            res = evaluate_tangent_normal_errors(T_true, T_est, N_true, N_est)
            tan_stats = res["tangent"]["stats"]
            norm_stats = res["normal"]["stats"]

            mean_tangent_deg[i, j] = tan_stats["mean_deg"]
            mean_normal_deg[i, j]  = norm_stats["mean_deg"]

            full_stats[(N, k)] = {
                "tangent": tan_stats,
                "normal": norm_stats,
            }

            print(f"N={N:4d}, k={k:3d} -> mean_tan={tan_stats['mean_deg']:.3f}°, "
                  f"mean_norm={norm_stats['mean_deg']:.3f}°")

    results = {
        "sample_sizes": sample_sizes,
        "k_values": k_values,
        "mean_tangent_deg": mean_tangent_deg,
        "mean_normal_deg": mean_normal_deg,
        "full_stats": full_stats,
    }
    return results


In [4]:
# sample_sizes = [200,300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000]
sample_sizes = np.linspace(150, 2000, 10, dtype=int).tolist()
k_values = np.linspace(2, 100, 25, dtype=int).tolist()
# k_values = [4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ]

results = run_spiral_knn_sweep(
    sample_sizes,
    k_values,
    a=0.5,
    b=0.2,
    t_min=0.0,
    t_max=4*np.pi,
    noise_sigma=0.02,  # or e.g. 0.02 to test robustness
)


N= 150, k=  2 -> mean_tan=7.067°, mean_norm=7.067°
N= 150, k=  6 -> mean_tan=2.118°, mean_norm=2.118°
N= 150, k= 10 -> mean_tan=2.685°, mean_norm=2.685°
N= 150, k= 14 -> mean_tan=4.458°, mean_norm=4.458°
N= 150, k= 18 -> mean_tan=5.900°, mean_norm=5.900°
N= 150, k= 22 -> mean_tan=7.798°, mean_norm=7.798°
N= 150, k= 26 -> mean_tan=14.295°, mean_norm=14.295°
N= 150, k= 30 -> mean_tan=19.130°, mean_norm=19.130°
N= 150, k= 34 -> mean_tan=22.709°, mean_norm=22.709°
N= 150, k= 38 -> mean_tan=26.168°, mean_norm=26.168°
N= 150, k= 42 -> mean_tan=29.077°, mean_norm=29.077°
N= 150, k= 46 -> mean_tan=30.314°, mean_norm=30.314°
N= 150, k= 51 -> mean_tan=30.310°, mean_norm=30.310°
N= 150, k= 55 -> mean_tan=29.956°, mean_norm=29.956°
N= 150, k= 59 -> mean_tan=29.275°, mean_norm=29.275°
N= 150, k= 63 -> mean_tan=28.696°, mean_norm=28.696°
N= 150, k= 67 -> mean_tan=26.493°, mean_norm=26.493°
N= 150, k= 71 -> mean_tan=25.979°, mean_norm=25.979°
N= 150, k= 75 -> mean_tan=24.848°, mean_norm=24.848°
N= 15

In [5]:
import plotly.express as px

def plot_heatmap_tangent_error(results, title="Mean Tangent Angle Error (deg)"):
    sample_sizes = results["sample_sizes"]
    k_values = results["k_values"]
    Z = results["mean_tangent_deg"]  # shape (len(N), len(k))

    fig = px.imshow(
        Z,
        x=k_values,
        y=sample_sizes,
        labels=dict(x="k (neighbors)", y="N (samples)", color="mean_deg"),
        title=title,
        aspect="auto",
    )
    fig.update_yaxes(type="category")  # nicer labels for N
    return fig

def plot_heatmap_normal_error(results, title="Mean Normal Angle Error (deg)"):
    sample_sizes = results["sample_sizes"]
    k_values = results["k_values"]
    Z = results["mean_normal_deg"]

    fig = px.imshow(
        Z,
        x=k_values,
        y=sample_sizes,
        labels=dict(x="k (neighbors)", y="N (samples)", color="mean_deg"),
        title=title,
        aspect="auto",
    )
    fig.update_yaxes(type="category")
    return fig


In [6]:
fig_tan = plot_heatmap_tangent_error(results)
fig_tan.show()

fig_norm = plot_heatmap_normal_error(results)
fig_norm.show()


In [7]:
import plotly.graph_objects as go

def plot_surface_tangent_error(results, title="Mean Tangent Angle Error (deg)"):
    sample_sizes = np.array(results["sample_sizes"])
    k_values = np.array(results["k_values"])
    Z = results["mean_tangent_deg"]  # shape (len(N), len(k))

    # Meshgrid so Plotly knows how to lay out the surface
    K, N = np.meshgrid(k_values, sample_sizes)

    fig = go.Figure(data=[
        go.Surface(
            x=K,
            y=N,
            z=Z,
            colorbar=dict(title="mean_deg"),
        )
    ])

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="k (neighbors)",
            yaxis_title="N (samples)",
            zaxis_title="mean tangent error (deg)",
        ),
    )
    return fig


In [8]:
def plot_surface_normal_error(results, title="Mean Normal Angle Error (deg)"):
    sample_sizes = np.array(results["sample_sizes"])
    k_values = np.array(results["k_values"])
    Z = results["mean_normal_deg"]

    K, N = np.meshgrid(k_values, sample_sizes)

    fig = go.Figure(data=[
        go.Surface(
            x=K,
            y=N,
            z=Z,
            colorbar=dict(title="mean_deg"),
        )
    ])

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="k (neighbors)",
            yaxis_title="N (samples)",
            zaxis_title="mean normal error (deg)",
        ),
    )
    return fig


In [9]:
fig = plot_surface_tangent_error(results)
fig.show()

fig = plot_surface_normal_error(results)
fig.show()


In [10]:
import numpy as np

def find_best_settings(results):
    sample_sizes = np.array(results["sample_sizes"])
    k_values = np.array(results["k_values"])

    # Matrices shape: (#sample_sizes, #k_values)
    Z_tan = results["mean_tangent_deg"]
    Z_norm = results["mean_normal_deg"]

    # --- Tangent ---
    idx_tan = np.unravel_index(np.argmin(Z_tan), Z_tan.shape)
    best_N_tan = sample_sizes[idx_tan[0]]
    best_k_tan = k_values[idx_tan[1]]
    best_err_tan = Z_tan[idx_tan]

    # --- Normal ---
    idx_norm = np.unravel_index(np.argmin(Z_norm), Z_norm.shape)
    best_N_norm = sample_sizes[idx_norm[0]]
    best_k_norm = k_values[idx_norm[1]]
    best_err_norm = Z_norm[idx_norm]

    results_summary = {
        "tangent": {
            "best_N": int(best_N_tan),
            "best_k": int(best_k_tan),
            "min_error_deg": float(best_err_tan),
            "matrix_idx": idx_tan,
        },
        "normal": {
            "best_N": int(best_N_norm),
            "best_k": int(best_k_norm),
            "min_error_deg": float(best_err_norm),
            "matrix_idx": idx_norm,
        },
    }

    return results_summary


In [11]:
best = find_best_settings(results)

print("Best Tangent Settings:")
print(best["tangent"])

print("\nBest Normal Settings:")
print(best["normal"])


Best Tangent Settings:
{'best_N': 2000, 'best_k': 67, 'min_error_deg': 1.0958666045185605, 'matrix_idx': (np.int64(9), np.int64(16))}

Best Normal Settings:
{'best_N': 2000, 'best_k': 67, 'min_error_deg': 1.0958666045179624, 'matrix_idx': (np.int64(9), np.int64(16))}
